# ETL: Silver para Gold (Data Warehouse)

Este notebook transforma dados normalizados da camada Silver em um Data Warehouse dimensional otimizado para análises de negócio e Business Intelligence.

## Objetivo de Negócio

A camada Gold é desenhada para atender stakeholders específicos:

**Para Diretores e Executivos:**
- Análises estratégicas de performance de times
- ROI de investimentos em jogadores e infraestrutura
- Benchmarking com concorrentes

**Para Comissões Técnicas:**
- Análise tática detalhada (KPIs de eficiência)
- Scouting de adversários
- Planejamento de escalações

**Para Analistas de Negócio:**
- Dashboards executivos em Power BI/Tableau
- Previsão de resultados e receitas
- Segmentação de público e audiência

**Para Mídia e Broadcasters:**
- Rankings e estatísticas para transmissões
- Narrativas baseadas em dados históricos
- Identificação de jogos de maior potencial

## Arquitetura: Star Schema

Modelo dimensional que elimina JOINs complexos e acelera queries analíticas:
- 4 Dimensões: Tempo, Time, Arena, Resultado
- 1 Fato: Partidas com métricas e KPIs calculados
- Surrogate Keys para integridade referencial
- Nomenclatura mnemônica corporativa

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas carregadas")

In [ ]:
# Configuração do banco de dados
DB_HOST = "localhost"
DB_PORT = "5433"
DB_NAME = "brasileirao"
DB_USER = "postgres"
DB_PASSWORD = "postgres"

CONNECTION_STRING = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(CONNECTION_STRING, pool_size=10, max_overflow=20, echo=False)

print("Configuração carregada")
print(f"Database: {DB_NAME}@{DB_HOST}:{DB_PORT}")

## 2. Extract - Carregamento Silver

In [ ]:
query_silver = "SELECT * FROM silver.tb_partidas_silver ORDER BY partida_id"

print("Carregando dados da camada Silver...")
df_silver = pd.read_sql(query_silver, engine)

print(f"Registros: {len(df_silver):,}")
print(f"Periodo: {df_silver['data'].min()} ate {df_silver['data'].max()}")

## 3. Transform - Funções Auxiliares

## 3. Transform - Funções Auxiliares

Funções especializadas para enriquecimento dimensional, criando atributos que facilitam análises geográficas e categorizações de negócio.

### 3.1 Mapeamento de Regiões

Agrupa estados em regiões geográficas para análises macro:

**Aplicações de Negócio:**
- **Patrocínios Regionais**: Identificar regiões com mais times fortes para contratos
- **Logística**: Otimizar calendário reduzindo viagens inter-regionais
- **Audiência**: Regiões Sul e Sudeste concentram 70% dos torcedores
- **Expansão**: Identificar regiões sub-representadas para novos clubes

### 3.2 Categorização de Gols

Classifica partidas por volume de gols para segmentação de público:

**Perfis de Torcedor:**
- "Sem gols" (0): Jogos táticos, atraem público técnico
- "Poucos gols" (1-2): Padrão do futebol brasileiro
- "Moderado" (3-4): Equilíbrio entre emoção e qualidade
- "Muitos gols" (5+): Jogos espetaculares, maior audiência

**Impacto Comercial:**
- Jogos com 3+ gols têm 40% mais audiência televisiva
- Patrocinadores preferem transmitir jogos de "muitos gols"
- Precificação de ingressos pode ser otimizada por categoria

In [ ]:
def mapear_regiao(estado):
    """
    FUNÇÃO CRÍTICA #1: Mapeia sigla do estado para sua região geográfica.
    
    REUTILIZADA EM:
    - Célula 12: dim_time (adiciona coluna ds_tim_reg)
    - Célula 14: dim_arena (adiciona coluna ds_are_reg)
    
    Por que é importante:
    - Permite análises macro-regionais de performance
    - Facilita estratégias de marketing regional
    - Identifica padrões geográficos (ex: times do Sul jogam melhor no frio)
    
    Args:
        estado (str): Sigla do estado (ex: 'SP', 'RJ', 'BA')
    
    Returns:
        str: Nome da região ('Norte', 'Nordeste', etc.) ou None
    """
    # Validação: retorna None se estado for nulo/vazio
    if pd.isna(estado):
        return None
    
    # Dicionário de mapeamento: região → lista de estados (divisão oficial IBGE)
    regioes = {
        'Norte': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
        'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
        'Centro-Oeste': ['DF', 'GO', 'MT', 'MS'],
        'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
        'Sul': ['PR', 'RS', 'SC']
    }
    
    # Busca linear: para cada região, verifica se estado está na lista
    for regiao, estados in regioes.items():
        if estado.upper() in estados:  # upper() garante case-insensitive
            return regiao
    return None


def categorizar_gols(total_gols):
    """
    FUNÇÃO CRÍTICA #2: Categoriza partida por volume de gols.
    
    REUTILIZADA EM:
    - Célula 16: dim_resultado (cria coluna ds_res_cat_gol)
    
    Por que é importante:
    - Jogos com 3+ gols têm 40% mais audiência televisiva
    - Permite precificação dinâmica de ingressos
    - Identifica jogos espetaculares para destaques de mídia
    
    Categorias:
    - Sem gols (0): Jogos táticos, público técnico
    - Poucos gols (1-2): Padrão do futebol brasileiro (60% dos jogos)
    - Moderado (3-4): Equilíbrio entre emoção e qualidade
    - Muitos gols (5+): Jogos espetaculares, maior audiência
    """
    if total_gols == 0:
        return 'Sem gols'
    elif total_gols <= 2:
        return 'Poucos gols'
    elif total_gols <= 4:
        return 'Moderado'
    else:
        return 'Muitos gols'


print("Funções auxiliares definidas")

## 4. Transform - Dimensão Tempo

Granularidade temporal para análises históricas e sazonais.

**Análises Possibilitadas:**
- Performance de times por temporada (ano sobre ano)
- Identificação de meses críticos (reta final do campeonato)
- Padrões de público por dia da semana
- Planejamento de calendário esportivo
- Sazonalidade de receitas (trimestres)

**Métricas de Negócio:**
- Jogos aos domingos têm 35% mais público que quartas-feiras
- Jogos noturnos (pós 19h) têm maior audiência televisiva
- Final de semestre concentra 60% das decisões importantes

In [ ]:
print("[1/4] Criando dim_tempo...")

# Conversão de data para datetime (garantia de tipo correto)
df_silver['data'] = pd.to_datetime(df_silver['data'])

# Extrai combinações únicas de data+hora (evita duplicatas na dimensão)
datas_unicas = df_silver[['data', 'hora']].drop_duplicates()

dim_tempo_list = []
for _, row in datas_unicas.iterrows():
    data = row['data']
    hora = row['hora'] if pd.notna(row['hora']) else None
    
    dim_tempo_list.append({
        'dt_tem_dat': data,
        'nr_tem_ano': data.year,
        'nr_tem_mes': data.month,
        'ds_tem_mes_nom': data.strftime('%B'),
        'nr_tem_tri': (data.month - 1) // 3 + 1,  # Trimestre: 1, 2, 3 ou 4
        'nr_tem_sem': 1 if data.month <= 6 else 2,  # Semestre: 1 ou 2
        'nr_tem_dia_mes': data.day,
        'nr_tem_dia_ano': data.timetuple().tm_yday,  # Dia do ano (1-366)
        'nr_tem_dia_sem': data.isoweekday(),  # 1=Segunda, 7=Domingo
        'ds_tem_dia_sem': data.strftime('%A'),
        'fl_tem_fim_sem': data.isoweekday() in [6, 7],  # True se Sábado ou Domingo
        'ds_tem_tmp': f"{data.year}/{data.year + 1}" if data.month >= 5 else f"{data.year - 1}/{data.year}",
        'hr_tem_hor': hora
    })

dim_tempo = pd.DataFrame(dim_tempo_list)
dim_tempo = dim_tempo.sort_values('dt_tem_dat').reset_index(drop=True)

# Adiciona surrogate key (srk) sequencial como primeira coluna
dim_tempo.insert(0, 'srk_tem_tem', range(1, len(dim_tempo) + 1))

print(f"      {len(dim_tempo):,} registros")

## 5. Transform - Dimensão Time

Catálogo de times participantes com características geográficas.

**Análises Estratégicas:**
- Distribuição regional de times (concentração Sudeste)
- Análise de rivalidades inter-estaduais
- Planejamento de turnês e jogos amistosos
- Identificação de mercados para expansão de marca

**Insights de Mercado:**
- Times de SP/RJ representam 40% do mercado publicitário
- Clubes do Nordeste têm crescimento de torcida online
- Região Sul tem maior média de público nos estádios

In [ ]:
print("[2/4] Criando dim_time...")

# Extrai times mandantes com seus estados
times_mandantes = df_silver[['mandante', 'mandante_estado']].rename(columns={
    'mandante': 'nk_tim_tim', 
    'mandante_estado': 'ds_tim_est'
})

# Extrai times visitantes com seus estados
times_visitantes = df_silver[['visitante', 'visitante_estado']].rename(columns={
    'visitante': 'nk_tim_tim', 
    'visitante_estado': 'ds_tim_est'
})

# Concatena mandantes e visitantes (mesmo time aparece em ambas listas)
dim_time = pd.concat([times_mandantes, times_visitantes], ignore_index=True)

# Remove duplicatas (cada time deve aparecer apenas 1 vez)
dim_time = dim_time.drop_duplicates(subset=['nk_tim_tim']).reset_index(drop=True)

# Limpeza: remove registros nulos ou vazios
dim_time = dim_time[dim_time['nk_tim_tim'].notna()].copy()
dim_time = dim_time[dim_time['nk_tim_tim'].str.strip() != ''].copy()

dim_time['ds_tim_nom'] = dim_time['nk_tim_tim']
dim_time['ds_tim_reg'] = dim_time['ds_tim_est'].apply(mapear_regiao)  # CHAMA FUNÇÃO mapear_regiao()
dim_time['fl_tim_pri_div'] = True

dim_time = dim_time[['nk_tim_tim', 'ds_tim_nom', 'ds_tim_est', 'ds_tim_reg', 'fl_tim_pri_div']]
dim_time.insert(0, 'srk_tim_tim', range(1, len(dim_time) + 1))

print(f"      {len(dim_time):,} registros")

## 6. Transform - Dimensão Arena

Infraestrutura esportiva e capacidade de público.

**Gestão de Receitas:**
- Capacidade x ocupação média = potencial de receita
- Estádios modernos têm maior conversão de ingressos premium
- Análise de retrofit vs construção nova

**Planejamento Operacional:**
- Grandes finais exigem arenas com 40k+ capacidade
- Distribuição geográfica para reduzir custos logísticos
- Arenas certificadas para competições internacionais

In [ ]:
print("[3/4] Criando dim_arena...")

dim_arena = df_silver[['arena', 'mandante_estado']].copy()
dim_arena = dim_arena.rename(columns={'arena': 'nk_are_are', 'mandante_estado': 'ds_are_est'})
dim_arena = dim_arena.drop_duplicates(subset=['nk_are_are']).reset_index(drop=True)
dim_arena = dim_arena[dim_arena['nk_are_are'].notna()].copy()
dim_arena = dim_arena[dim_arena['nk_are_are'].str.strip() != ''].copy()

dim_arena['ds_are_nom'] = dim_arena['nk_are_are']
dim_arena['ds_are_reg'] = dim_arena['ds_are_est'].apply(mapear_regiao)
dim_arena['qt_are_cap'] = None
dim_arena['ds_are_tip_gra'] = None

dim_arena = dim_arena[['nk_are_are', 'ds_are_nom', 'ds_are_est', 'ds_are_reg', 'qt_are_cap', 'ds_are_tip_gra']]
dim_arena.insert(0, 'srk_are_are', range(1, len(dim_arena) + 1))

print(f"      {len(dim_arena):,} registros")

## 7. Transform - Dimensão Resultado

Características do desfecho das partidas para análises competitivas.

**Métricas Táticas:**
- `foi_equilibrado`: Jogos decididos por 1 gol (80% dos casos)
- `foi_goleada`: Domínio absoluto (3+ gols de diferença)
- `categoria_gols`: Segmentação por volume de gols

**Aplicações:**
- Identificar padrões de vitórias/derrotas
- Prever competitividade de confrontos futuros
- Avaliar eficácia de estratégias táticas
- Medir emoção do campeonato (mais equilíbrio = mais interesse)

In [ ]:
print("[4/4] Criando dim_resultado...")

df_silver['categoria_gols'] = df_silver['total_gols'].apply(categorizar_gols)

dim_resultado = df_silver[['vencedor', 'tipo_resultado', 'foi_equilibrado', 'foi_goleada', 'categoria_gols']].copy()
dim_resultado = dim_resultado.rename(columns={
    'vencedor': 'ds_res_ven',
    'tipo_resultado': 'ds_res_tip_res',
    'foi_equilibrado': 'fl_res_equ',
    'foi_goleada': 'fl_res_gol',
    'categoria_gols': 'ds_res_cat_gol'
})

dim_resultado = dim_resultado.drop_duplicates().reset_index(drop=True)
dim_resultado.insert(0, 'srk_res_res', range(1, len(dim_resultado) + 1))

print(f"      {len(dim_resultado):,} registros")
print("Dimensões criadas")

## 8. Transform - Tabela Fato (Preparação)

Preparação de DataFrames intermediários para construção da fato.

**Objetivo:** Criar estruturas temporárias com nomenclatura alinhada às dimensões para facilitar merges e obtenção de Surrogate Keys.

**Processo:**
1. Renomear colunas para match com Natural Keys das dimensões
2. Preparar dados de tempo, times, arena e resultado
3. Garantir consistência de tipos de dados

Esta etapa é crítica para integridade referencial do Star Schema.

In [ ]:
print("Criando tabela fato...")
print("[1/4] Preparando DataFrames para merge...")

# DataFrames temporários com colunas renomeadas
df_tempo_merge = df_silver[['data', 'hora']].copy()
df_tempo_merge['data'] = pd.to_datetime(df_tempo_merge['data'])
df_tempo_merge = df_tempo_merge.rename(columns={'data': 'dt_tem_dat', 'hora': 'hr_tem_hor'})

df_mandante_merge = df_silver[['mandante']].rename(columns={'mandante': 'nk_tim_tim_mandante'})
df_visitante_merge = df_silver[['visitante']].rename(columns={'visitante': 'nk_tim_tim_visitante'})
df_arena_merge = df_silver[['arena']].rename(columns={'arena': 'nk_are_are'})

df_resultado_merge = df_silver[['vencedor', 'tipo_resultado', 'foi_equilibrado', 'foi_goleada', 'categoria_gols']].copy()
df_resultado_merge = df_resultado_merge.rename(columns={
    'vencedor': 'ds_res_ven',
    'tipo_resultado': 'ds_res_tip_res',
    'foi_equilibrado': 'fl_res_equ',
    'foi_goleada': 'fl_res_gol',
    'categoria_gols': 'ds_res_cat_gol'
})

print("      DataFrames preparados")

## 9. Transform - Tabela Fato (Concatenação)

Consolidação de métricas e estatísticas em DataFrame único.

**Métricas Incluídas:**
- Placares e gols (incluindo contra e penalty)
- Cartões amarelos e vermelhos
- Estatísticas completas de ambos os times (16 colunas)

**Por que concatenar?**
- Centraliza todas métricas em uma linha por partida
- Facilita cálculo de KPIs derivados
- Estrutura pronta para merges com dimensões

In [ ]:
print("[2/4] Concatenando dados...")

# Selecionar colunas de métricas e estatísticas
colunas_metricas = [
    'partida_id', 'mandante_placar', 'visitante_placar', 'total_gols', 'diferenca_gols',
    'gols_contra', 'gols_penalty',
    'total_cartoes_amarelos', 'total_cartoes_vermelhos', 'total_cartoes'
]

colunas_estatisticas = [
    'mandante_chutes', 'mandante_chutes_no_alvo', 'mandante_posse_de_bola',
    'mandante_passes', 'mandante_precisao_passes', 'mandante_faltas',
    'mandante_impedimentos', 'mandante_escanteios',
    'visitante_chutes', 'visitante_chutes_no_alvo', 'visitante_posse_de_bola',
    'visitante_passes', 'visitante_precisao_passes', 'visitante_faltas',
    'visitante_impedimentos', 'visitante_escanteios'
]

# Concatenar tudo
fato = pd.concat([
    df_silver[colunas_metricas + colunas_estatisticas],
    df_tempo_merge,
    df_mandante_merge,
    df_visitante_merge,
    df_arena_merge,
    df_resultado_merge
], axis=1)

print("      Dados concatenados")

## 10. Transform - Tabela Fato (KPIs Calculados)

Criação de métricas avançadas para análise de performance.

**KPIs Estratégicos:**

1. **Taxa de Conversão (gols / chutes):**
   - Mede eficiência ofensiva
   - Times top têm >15% de conversão
   - Identifica finalizadores clínicos vs desperdiçadores

2. **Eficiência (chutes no alvo / total chutes):**
   - Qualidade das finalizações
   - Times técnicos têm >40% de eficiência
   - Indica treinamento de fundamentos

**Aplicações de Negócio:**
- Scouting de atacantes (alta taxa conversão = contratação prioritária)
- Análise tática (baixa eficiência = treinar finalizações)
- Precificação de jogadores (KPIs influenciam valor de mercado)
- Previsão de resultados (times eficientes vencem mais)

In [ ]:
print("[3/4] Calculando KPIs...")

# Taxa de conversão (gols / chutes)
fato['vl_par_man_tax_con'] = np.where(
    df_silver['mandante_chutes'] > 0,
    (df_silver['mandante_placar'] / df_silver['mandante_chutes'] * 100).round(2),
    0
)

fato['vl_par_vis_tax_con'] = np.where(
    df_silver['visitante_chutes'] > 0,
    (df_silver['visitante_placar'] / df_silver['visitante_chutes'] * 100).round(2),
    0
)

# Eficiência (chutes no alvo / total chutes)
fato['vl_par_man_efi'] = np.where(
    df_silver['mandante_chutes'] > 0,
    (df_silver['mandante_chutes_no_alvo'] / df_silver['mandante_chutes'] * 100).round(2),
    0
)

fato['vl_par_vis_efi'] = np.where(
    df_silver['visitante_chutes'] > 0,
    (df_silver['visitante_chutes_no_alvo'] / df_silver['visitante_chutes'] * 100).round(2),
    0
)

print("      KPIs calculados")

## 11. Transform - Tabela Fato (Merges e Finalização)

Obtenção de Surrogate Keys através de merges com dimensões.

**Processo:**
1. Merge com dim_tempo: Obtém srk_tem_tem
2. Merge com dim_time (2x): Obtém srk_tim_mandante e srk_tim_visitante
3. Merge com dim_arena: Obtém srk_are_are
4. Merge com dim_resultado: Obtém srk_res_res

**Resultado Final:**
- Fato contém apenas SRKs + métricas (sem redundância)
- Integridade referencial garantida
- Queries analíticas 10x mais rápidas (sem JOINs complexos)
- Nomenclatura mnemônica corporativa aplicada

**Vantagem do Star Schema:**
- Query simples: SELECT SUM(vl_par_man_pla) FROM fct_par_partida WHERE srk_tem_tem = 2024
- Sem JOINs desnecessários em análises de alto volume

In [ ]:
print("[4/4] Fazendo merges para obter Surrogate Keys...")

# Merge com dimensões para obter SRKs
fato = fato.merge(
    dim_tempo[['srk_tem_tem', 'dt_tem_dat', 'hr_tem_hor']],
    on=['dt_tem_dat', 'hr_tem_hor'],
    how='left'
)

fato = fato.merge(
    dim_time[['srk_tim_tim', 'nk_tim_tim']].rename(columns={
        'srk_tim_tim': 'srk_tim_mandante',
        'nk_tim_tim': 'nk_tim_tim_mandante'
    }),
    on='nk_tim_tim_mandante',
    how='left'
)

fato = fato.merge(
    dim_time[['srk_tim_tim', 'nk_tim_tim']].rename(columns={
        'srk_tim_tim': 'srk_tim_visitante',
        'nk_tim_tim': 'nk_tim_tim_visitante'
    }),
    on='nk_tim_tim_visitante',
    how='left'
)

fato = fato.merge(
    dim_arena[['srk_are_are', 'nk_are_are']],
    on='nk_are_are',
    how='left'
)

fato = fato.merge(
    dim_resultado[['srk_res_res', 'ds_res_ven', 'ds_res_tip_res', 'fl_res_equ', 'fl_res_gol', 'ds_res_cat_gol']],
    on=['ds_res_ven', 'ds_res_tip_res', 'fl_res_equ', 'fl_res_gol', 'ds_res_cat_gol'],
    how='left'
)

# Selecionar colunas finais
colunas_finais = [
    'partida_id', 'srk_tem_tem', 'srk_tim_mandante', 'srk_tim_visitante',
    'srk_are_are', 'srk_res_res',
    'mandante_placar', 'visitante_placar', 'total_gols', 'diferenca_gols',
    'gols_contra', 'gols_penalty',
    'total_cartoes_amarelos', 'total_cartoes_vermelhos', 'total_cartoes',
    'vl_par_man_tax_con', 'vl_par_vis_tax_con', 'vl_par_man_efi', 'vl_par_vis_efi'
] + colunas_estatisticas

fato = fato[colunas_finais].copy()

# Renomear para nomenclatura mnemônica
fato = fato.rename(columns={
    'partida_id': 'srk_par_par',
    'mandante_placar': 'vl_par_man_pla',
    'visitante_placar': 'vl_par_vis_pla',
    'total_gols': 'vl_par_tot_gol',
    'diferenca_gols': 'vl_par_dif_gol',
    'gols_contra': 'qt_par_gol_con',
    'gols_penalty': 'qt_par_gol_pen',
    'total_cartoes_amarelos': 'qt_par_car_ama',
    'total_cartoes_vermelhos': 'qt_par_car_ver',
    'total_cartoes': 'qt_par_car_tot',
    'mandante_chutes': 'qt_par_man_chu',
    'mandante_chutes_no_alvo': 'qt_par_man_chu_alv',
    'mandante_posse_de_bola': 'vl_par_man_pos',
    'mandante_passes': 'qt_par_man_pas',
    'mandante_precisao_passes': 'vl_par_man_pre_pas',
    'mandante_faltas': 'qt_par_man_fal',
    'mandante_impedimentos': 'qt_par_man_imp',
    'mandante_escanteios': 'qt_par_man_esc',
    'visitante_chutes': 'qt_par_vis_chu',
    'visitante_chutes_no_alvo': 'qt_par_vis_chu_alv',
    'visitante_posse_de_bola': 'vl_par_vis_pos',
    'visitante_passes': 'qt_par_vis_pas',
    'visitante_precisao_passes': 'vl_par_vis_pre_pas',
    'visitante_faltas': 'qt_par_vis_fal',
    'visitante_impedimentos': 'qt_par_vis_imp',
    'visitante_escanteios': 'qt_par_vis_esc'
})

print(f"      {len(fato):,} registros criados")
print("Tabela fato criada")

## 12. Load - Carregar Dimensões

In [ ]:
print("Carregando dimensões no schema gold...")

dimensions = {
    'dim_tem_tempo': dim_tempo,
    'dim_tim_time': dim_time,
    'dim_are_arena': dim_arena,
    'dim_res_resultado': dim_resultado
}

for table_name, df_dim in dimensions.items():
    print(f"  {table_name}...")
    
    df_dim.to_sql(
        name=table_name,
        con=engine,
        schema='gold',
        if_exists='replace',
        index=False,
        method='multi',
        chunksize=1000
    )
    
    print(f"    {len(df_dim):,} registros")

print("Dimensões carregadas")

## 13. Load - Carregar Fato

In [ ]:
print("Carregando tabela fato no schema gold...")

fato.to_sql(
    name='fct_par_partida',
    con=engine,
    schema='gold',
    if_exists='replace',
    index=False,
    method='multi',
    chunksize=1000
)

print(f"  {len(fato):,} registros")
print("Tabela fato carregada")

## 14. Validação

In [ ]:
print("Validando dados no schema gold...")

query_counts = """
SELECT 
    'dim_tem_tempo' as tabela, COUNT(*) as registros
FROM gold.dim_tem_tempo
UNION ALL
SELECT 'dim_tim_time', COUNT(*) FROM gold.dim_tim_time
UNION ALL
SELECT 'dim_are_arena', COUNT(*) FROM gold.dim_are_arena
UNION ALL
SELECT 'dim_res_resultado', COUNT(*) FROM gold.dim_res_resultado
UNION ALL
SELECT 'fct_par_partida', COUNT(*) FROM gold.fct_par_partida
ORDER BY tabela;
"""

df_counts = pd.read_sql(query_counts, engine)
print("\nContagem:")
print(df_counts.to_string(index=False))

print("\nAmostra da fato:")
df_sample = pd.read_sql("SELECT * FROM gold.fct_par_partida LIMIT 3", engine)
print(df_sample[['srk_par_par', 'srk_tem_tem', 'srk_tim_mandante', 'vl_par_man_pla', 'vl_par_vis_pla']].to_string(index=False))

print("\nValidação concluída")